In [ ]:
  !pip install langchain-community langchain-text-splitters  pypdf  langchain-huggingface sentence-transformers faiss-cpu ollama langchain-ollama   --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 90.8 MB/s eta 0:00:00


In [ ]:
# 第一個 Cell
!pip install -q bitsandbytes langchain-huggingface sentencepiece
!pip install -q -U transformers accelerate
# 注意：這裡刻意不寫 torch，直接用 Colab 內建的，就不會打架也不會慢

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 50.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.3/553.3 kB 37.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.4.1 which is incompatible.


In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="Qwen/Qwen3-Embedding-0.6B",
    model_kwargs={'device': 'cuda'},
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

In [ ]:
from typing import List,Tuple,Any
import numpy as np
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from scipy.special import logsumexp
import faiss

In [ ]:
chunk_size=1024
chunk_overlap=128
path=r"/content/drive/MyDrive/RAG/RAG資料集/2401.15884v3.pdf"
query="What the CRAG proposed to improve "

In [ ]:
def replace_tap_with_space(chunks):
    for chunk in chunks:
        chunk.page_content=chunk.page_content.replace("\t"," ")
    return chunks

def encode_pdf(path,chunk_size,chunk_overlap):
    loader=PyPDFLoader(path)
    documents=loader.load()
    documents=documents * 3 #為了模擬很多資料重複
    splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    chunks=splitter.split_documents(documents)

    cleaned_chunks=replace_tap_with_space(chunks)

    vectorstore=FAISS.from_documents(cleaned_chunks,embedding_model)

    return vectorstore


In [ ]:
vectorstore=encode_pdf(path,chunk_size,chunk_overlap)

In [ ]:
def idx_to_text(idx: int):
    docstore_id=vectorstore.index_to_docstore_id[idx]
    document=vectorstore.docstore.search(docstore_id)
    return document.page_content

def get_context(query:str ,k:int) ->List[str]:
    q_vec=vectorstore.embedding_function.embed_query(query)
    _, indices = vectorstore.index.search(np.array([q_vec]), k=k)

    texts=[idx_to_text(i) for i in indices[0]]

    return texts

def show_context(context):
    """
    Display the contents of the provided context list.

    Args:
        context (list): A list of context items to be displayed.

    Prints each context item in the list with a heading indicating its position.
    """
    for i, c in enumerate(context):
        print(f"Context {i + 1}:")
        print(c)
        print("\n")



In [ ]:
texts=get_context(query,k=3)
show_context(texts)

Context 1:
knowledge retrieval actions discriminately. With
the further leverage of web search and optimized
knowledge utilization, CRAG has significantly
improved the ability of automatic self-correction
and efficient utilization of retrieved documents.
Experiments extensively demonstrate its adaptabil-
ity to RAG-based approaches as well as general-
izability across short- and long-form generation
tasks. While we primarily proposed to improve the
RAG framework from a corrective perspective and
CRAG can be seamlessly coupled with various
RAG-based approaches, fine-tuning an external
retrieval evaluator is inevitable. How to eliminate
this external evaluator and equip LLMs with better
retrieval evaluation capabilities will be our future
work.
References
Rohan Anil, Andrew M. Dai, Orhan Firat, Melvin John-
son, Dmitry Lepikhin, Alexandre Passos, Siamak
Shakeri, Emanuel Taropa, Paige Bailey, Zhifeng
Chen, Eric Chu, Jonathan H. Clark, Laurent El
Shafey, Yanping Huang, Kathy Meier-Hellster

In [ ]:
def lognorm(dist:np.ndarray, sigma:float):
    if sigma < 1e-9:
        return -np.inf * dist
    return -np.log(sigma) - 0.5 * np.log(2 * np.pi) - dist**2 / (2 * sigma**2)


In [ ]:
deversity_weight=1.0
relevance_weight=1.0
sigma=0.1

def greedy_dartsearch(
    query_distance : np.ndarray,
    document_distance : np.ndarray,
    documents : List[Document],
    num_results : int,
) -> Tuple[List[str],List[float]]:

    query_prob=lognorm(query_distance,sigma)
    document_prob=lognorm(document_distance,sigma)

    most_relevant_idx=np.argmax(query_prob) #返回陣列中最大值的索引（index）
    selected_indices=np.array([most_relevant_idx])
    selected_score=[1.0]

    max_distance=document_prob[most_relevant_idx] #第一個選的文章

    while len(selected_indices) < num_results:
        update_distance=np.maximum(max_distance,document_prob)

        combined_scores = (update_distance * deversity_weight + query_prob * relevance_weight)

        normalized_scores = logsumexp(combined_scores, axis=1)
        normalized_scores[selected_indices] = -np.inf

        best_idx=np.argmax(normalized_scores)
        best_score=np.max(normalized_scores)

        max_distance=update_distance[best_idx]
        selected_indices=np.append(selected_indices,best_idx)
        selected_score.append(best_score)

    selected_documents=[documents[i] for i in selected_indices]
    return selected_documents,selected_score


In [ ]:
def get_context_with_dartboard(
    query:str,
    num_results:int,
    oversampling_factor: int = 3
)->Tuple[List[str],List[float]]:

    query_embedding=vectorstore.embedding_function.embed_query(query)
    _,candidate_indices=vectorstore.index.search(np.array([query_embedding]),k=num_results * oversampling_factor) #

    candidate_vectors = np.array([
    vectorstore.index.reconstruct(int(idx))
    for idx in candidate_indices[0]
    ])
    candidate_texts=[idx_to_text(idx) for idx in candidate_indices[0]]

    document_distance = 1 -np.dot(candidate_vectors,candidate_vectors.T)
    query_distance=1 - np.dot(query_embedding,candidate_vectors.T)

    selected_texts,selected_scores=greedy_dartsearch(
        query_distance,
        document_distance,
        candidate_texts,
        num_results
    )
    return selected_texts,selected_scores


In [ ]:
texts,scores=get_context_with_dartboard(query,num_results=3)
show_context(texts)

Context 1:
knowledge retrieval actions discriminately. With
the further leverage of web search and optimized
knowledge utilization, CRAG has significantly
improved the ability of automatic self-correction
and efficient utilization of retrieved documents.
Experiments extensively demonstrate its adaptabil-
ity to RAG-based approaches as well as general-
izability across short- and long-form generation
tasks. While we primarily proposed to improve the
RAG framework from a corrective perspective and
CRAG can be seamlessly coupled with various
RAG-based approaches, fine-tuning an external
retrieval evaluator is inevitable. How to eliminate
this external evaluator and equip LLMs with better
retrieval evaluation capabilities will be our future
work.
References
Rohan Anil, Andrew M. Dai, Orhan Firat, Melvin John-
son, Dmitry Lepikhin, Alexandre Passos, Siamak
Shakeri, Emanuel Taropa, Paige Bailey, Zhifeng
Chen, Eric Chu, Jonathan H. Clark, Laurent El
Shafey, Yanping Huang, Kathy Meier-Hellster